In [26]:
%cd /home/jovyan/onceuponai/websites/src/stories-fantasy-shop-assistant/notebooks
from IPython.core.display import HTML
HTML("""
<link rel="stylesheet" type="text/css" href="//fonts.googleapis.com/css?family=Bangers" />
<style>
h1 {font-family: Bangers !important; font-size: 150px !important}
h2 {font-family: Bangers !important; font-size: 120px !important}
h3 {font-family: Bangers !important; font-size: 100px !important}
h4 {font-family: Bangers !important; font-size: 80px !important}
h5 {font-family: Bangers !important; font-size: 50px !important}
h6 {font-family: Bangers !important; font-size: 30px !important}
.txt7 {font-family: Bangers !important; font-size: 70px !important}
.txt6 {font-family: Bangers !important; font-size: 60px !important}
.txt5 {font-family: Bangers !important; font-size: 50px !important}
.txt4 {font-family: Bangers !important; font-size: 40px !important}
.txt3 {font-family: Bangers !important; font-size: 30px !important}
.cap {font-family: Bangers !important; font-size: 30px !important}
.controls
{
    display: none !important;
}

.slide-number
{
    display: none !important;
}
.inline-block {
   display: inline-block;
}

.backgrounds {
   background-color: #fff !important;
}

.controls {
  display: none !important;
}
</style>
""")

import pandas as pd
pd.set_option('display.max_colwidth', None)
import warnings
warnings.filterwarnings('ignore')

/home/jovyan/onceuponai/websites/src/stories-fantasy-shop-assistant/notebooks


In [ ]:
!pip install deltalake polars

## Prepare data

In [ ]:
import yaml
from deltalake.writer import write_deltalake
import pandas as pd
import polars as pl

data = yaml.safe_load(open("recipes.yaml", "r"))

prep={}


df = pl.DataFrame(data["Recipes"])
df.to_pandas()

# Embeddings

In [1]:
import adventures

a = adventures.Adventures("intfloat/multilingual-e5-small")

In [2]:
a.embeddings(["witaj przygodo"])

[[0.03098888322710991,
  -0.01199488528072834,
  -0.07422228902578354,
  -0.06891388446092606,
  0.05637747794389725,
  -0.03292524069547653,
  0.05126916617155075,
  0.03614596650004387,
  0.023003606125712395,
  0.006733578629791737,
  0.034447889775037766,
  0.02289721742272377,
  0.05806450918316841,
  -0.04183975234627724,
  -0.02903415448963642,
  0.03861762955784798,
  0.06157746538519859,
  -0.06715274602174759,
  -0.038856398314237595,
  -0.029826771467924118,
  0.039446957409381866,
  -0.03411424532532692,
  -0.06606124341487885,
  0.04624686390161514,
  0.05065515637397766,
  0.03813409432768822,
  -0.0525520034134388,
  0.06392091512680054,
  0.037307772785425186,
  -0.05885927751660347,
  -0.05158555135130882,
  -0.05430685728788376,
  0.03010113537311554,
  -0.03858902305364609,
  0.05888715386390686,
  0.08750040084123611,
  -0.02383234351873398,
  -0.03985855355858803,
  0.0594383180141449,
  -0.05335207283496857,
  -0.009211460128426552,
  0.019977519288659096,
  0.020

In [ ]:
items = []
for combined_text in df['Recipe']:
    emb = a.embeddings([combined_text])
    items.append({"item": combined_text, "vector": emb[0]})

items

# Fake words

In [ ]:
import random

subjects = [
    "The mischievous squirrel", "The colorful birds", "The playful puppies", 
    "The dedicated teacher", "The curious boy", "The brilliant inventor", "Girl", "Cat", "Mouse", "Dog"
]

verbs = [
    "danced", "explored", "built", "discovered", "imagined", 
    "sang", "cooked", "programmed","run", "sing", "walk", 
]

objects = ["the sandwich", "a brilliant app", "over the fence", "a beautiful song", "in the sky"]

for i in range(300):
    phrase = random.choice(subjects) + " " + random.choice(verbs) + " " + random.choice(objects)
    emb = a.embeddings([phrase])
    items.append({"item": phrase, "vector": emb[0]})

# Lance DB

In [ ]:
import os
import getpass

print("ACCOUNT NAME")
os.environ["AZURE_STORAGE_ACCOUNT_NAME"]=getpass.getpass()

print("ACCOUNT KEY")
os.environ["AZURE_STORAGE_ACCOUNT_KEY"]=getpass.getpass()

In [ ]:
import lancedb

db = await lancedb.connect_async("az://recipesv2/vectors2")
tbl = await db.create_table("recipes_vectors", data=items)

In [ ]:
res = await tbl.create_index("vector")

# Search

In [ ]:
import lancedb

emb = a.embeddings(["Chciałbym zrobić żurek"])

tbl = await db.open_table("recipes_vectors")
res = await tbl.query().nearest_to(emb[0]).limit(1).to_pandas()
res